# <a href="https://colab.research.google.com/github/oftx/python-sstv/blob/main/SSTV_Video_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SSTV Video Glitch Generator

This notebook runs the `python-sstv` tool suite to convert video to SSTV signal and decode it back, apply random glitch effects (Noise, Jitter, Skew), and simulated analog artifacts.

### Features
- **Video to SSTV**: Convert MP4 video to SSTV audio and back to frames.
- **Glitch Effects**: Phase Jitter (Shake), Clock Drift (Slant/Wobble), Line Tearing.
- **Customizable**: Adjust bitrate, resolution presets, and randomness seeds.


In [ ]:
#@title 1. Install Dependencies
!pip install opencv-python-headless scipy numpy pillow tqdm
!apt-get install ffmpeg

import os
import sys
print("Dependencies installed.")

## 2. Deploy Code
Writing project files to Colab environment.

In [ ]:
%%writefile common.py

import numpy as np

# Robot36 Constants (Base frequencies)
SAMPLE_RATE = 48000

# Frequencies
SYNC_FREQ = 1200.0
PORCH_FREQ = 1500.0
VIS_BIT_1_FREQ = 1100.0
VIS_BIT_0_FREQ = 1300.0
BLACK_FREQ = 1500.0
WHITE_FREQ = 2300.0
EVEN_SEPARATOR_FREQ = 1500.0
ODD_SEPARATOR_FREQ = 2300.0
PORCH_SEPARATOR_FREQ = 1900.0
LEADER_TONE_FREQ = 1900.0
BREAK_FREQ = 1200.0

# Base Durations (ms) for Robot36 Standard (320x240)
SYNC_DURATION_MS = 9.0
SYNC_PORCH_DURATION_MS = 3.0
SEPARATOR_DURATION_MS = 4.5
PORCH_DURATION_MS = 1.5
LEADER_TONE_DURATION_MS = 300.0
BREAK_DURATION_MS = 10.0
VIS_BIT_DURATION_MS = 30.0

def ms_to_samples(ms, sample_rate=SAMPLE_RATE):
    return int(round(ms * sample_rate / 1000.0))

class ModeConfig:
    def __init__(self, width=320, height=240, sample_rate=SAMPLE_RATE):
        self.width = width
        self.height = height
        self.sample_rate = sample_rate
        
        # Calculate scan durations based on width
        # Standard Robot36 (320px) takes 88ms for Y and 44ms for UV
        # We scale this linearly
        self.y_scan_duration_ms = 88.0 * (width / 320.0)
        self.uv_scan_duration_ms = 44.0 * (width / 320.0)
        
        # Sample counts
        self.sync_samples = ms_to_samples(SYNC_DURATION_MS, sample_rate)
        self.sync_porch_samples = ms_to_samples(SYNC_PORCH_DURATION_MS, sample_rate)
        self.separator_samples = ms_to_samples(SEPARATOR_DURATION_MS, sample_rate)
        self.porch_samples = ms_to_samples(PORCH_DURATION_MS, sample_rate)
        
        self.y_scan_samples = ms_to_samples(self.y_scan_duration_ms, sample_rate)
        self.uv_scan_samples = ms_to_samples(self.uv_scan_duration_ms, sample_rate)

        # Header samples (fixed)
        self.leader_tone_samples = ms_to_samples(LEADER_TONE_DURATION_MS, sample_rate)
        self.break_samples = ms_to_samples(BREAK_DURATION_MS, sample_rate)
        self.vis_bit_samples = ms_to_samples(VIS_BIT_DURATION_MS, sample_rate)

def get_mode_config(width, height, sample_rate=SAMPLE_RATE):
    return ModeConfig(width, height, sample_rate)

def rgb_to_yuv(r, g, b):
    # Rec. 601 limited range as per Java implementation
    # Y = 16 + (65.738*R + 129.057*G + 25.064*B) / 256
    # U = 128 + (-37.945*R - 74.494*G + 112.439*B) / 256
    # V = 128 + (112.439*R - 94.154*G - 18.285*B) / 256
    
    y = 16.0 + (65.738 * r + 129.057 * g + 25.064 * b) / 256.0
    u = 128.0 + (-37.945 * r - 74.494 * g + 112.439 * b) / 256.0
    v = 128.0 + (112.439 * r - 94.154 * g - 18.285 * b) / 256.0
    
    return np.clip(y, 16, 235), np.clip(u, 16, 240), np.clip(v, 16, 240)

def yuv_to_rgb(y, u, v):
    # Inverse of the above
    # Y -= 16
    # U -= 128
    # V -= 128
    # R = (298 * Y + 409 * V + 128) >> 8
    # G = (298 * Y - 100 * U - 208 * V + 128) >> 8
    # B = (298 * Y + 516 * U + 128) >> 8
    
    y_shifted = y - 16.0
    u_shifted = u - 128.0
    v_shifted = v - 128.0
    
    r = (298.082 * y_shifted + 408.583 * v_shifted) / 256.0
    g = (298.082 * y_shifted - 100.291 * u_shifted - 208.120 * v_shifted) / 256.0
    b = (298.082 * y_shifted + 516.412 * u_shifted) / 256.0
    
    return np.clip(r, 0, 255), np.clip(g, 0, 255), np.clip(b, 0, 255)



In [ ]:
%%writefile encoder.py

import numpy as np
import math
import argparse
from PIL import Image
from scipy.io import wavfile
import common

class SSTVEncoder:
    def __init__(self, image_source, width=320, height=240, sample_rate=common.SAMPLE_RATE):
        self.image_source = image_source
        self.config = common.get_mode_config(width, height, sample_rate)
        self.phase = 0.0
        self.audio_buffer = []

    def load_and_process_image(self):
        if isinstance(self.image_source, str):
            img = Image.open(self.image_source).convert('RGB')
        elif isinstance(self.image_source, Image.Image):
            img = self.image_source.convert('RGB')
        elif isinstance(self.image_source, np.ndarray):
            img = Image.fromarray(self.image_source).convert('RGB')
        else:
            raise ValueError("Unsupported image source type")
            
        img = img.resize((self.config.width, self.config.height), Image.Resampling.LANCZOS)
        return np.array(img)

    def add_tone(self, freq, duration_samples):
        # Generate tone with continuous phase
        t = np.arange(duration_samples)
        phase_increment = 2 * np.pi * freq / self.config.sample_rate
        phases = self.phase + phase_increment * t
        
        # Update phase for next segment
        self.phase = phases[-1] + phase_increment 
        
        tone = np.sin(phases)
        self.audio_buffer.append(tone)

    def add_scan_line_tone(self, pixels, duration_samples, channel_idx):
        # pixels: array of pixel values for the channel
        # Map pixel values to frequency range 1500Hz - 2300Hz
        
        freqs = common.BLACK_FREQ + (pixels * (common.WHITE_FREQ - common.BLACK_FREQ) / 255.0)
        
        # Generate samples with varying frequency (FM)
        t_indices = np.linspace(0, len(pixels) - 1, duration_samples)
        interpolated_freqs = np.interp(t_indices, np.arange(len(pixels)), freqs)
        
        # Integrate frequency to get phase
        # phase[n] = phase[n-1] + 2*pi*f[n]/Fs
        phase_increments = 2 * np.pi * interpolated_freqs / self.config.sample_rate
        cumulative_phases = np.cumsum(phase_increments) + self.phase
        
        self.phase = cumulative_phases[-1]
        
        tone = np.sin(cumulative_phases)
        self.audio_buffer.append(tone)

    def generate_header(self):
        # Calibration Header
        # Leader tone
        self.add_tone(common.LEADER_TONE_FREQ, self.config.leader_tone_samples)
        # Break
        self.add_tone(common.BREAK_FREQ, self.config.break_samples)
        # Leader tone
        self.add_tone(common.LEADER_TONE_FREQ, self.config.leader_tone_samples)
        # VIS Start bit
        self.add_tone(common.BREAK_FREQ, self.config.vis_bit_samples)
        
        # VIS Code 8 (Robot36): 00001000 (LSB first) -> 0, 0, 0, 1, 0, 0, 0, 0
        vis_code = 8
        parity = 0
        for i in range(7):
            bit = (vis_code >> i) & 1
            parity ^= bit
            freq = common.VIS_BIT_1_FREQ if bit else common.VIS_BIT_0_FREQ
            self.add_tone(freq, self.config.vis_bit_samples)
            
        # Parity bit
        freq = common.VIS_BIT_1_FREQ if parity else common.VIS_BIT_0_FREQ
        self.add_tone(freq, self.config.vis_bit_samples)
        
        # Stop bit
        self.add_tone(common.BREAK_FREQ, self.config.vis_bit_samples)

    def write_wav(self, file_object):
        img_array = self.load_and_process_image()
        
        # Convert to YUV
        r = img_array[:,:,0].astype(float)
        g = img_array[:,:,1].astype(float)
        b = img_array[:,:,2].astype(float)
        
        y, u, v = common.rgb_to_yuv(r, g, b)
        
        # Generate Audio
        self.generate_header()
        
        for line in range(self.config.height):
            # Sync
            self.add_tone(common.SYNC_FREQ, self.config.sync_samples)
            # Sync Porch
            self.add_tone(common.PORCH_FREQ, self.config.sync_porch_samples)
            
            # Y Scan
            self.add_scan_line_tone(y[line], self.config.y_scan_samples, 0)
            
            if line % 2 == 0:
                # Even Line: Separator (1500) -> Porch -> V Scan
                self.add_tone(common.EVEN_SEPARATOR_FREQ, self.config.separator_samples)
                self.add_tone(common.PORCH_SEPARATOR_FREQ, self.config.porch_samples)
                self.add_scan_line_tone(v[line], self.config.uv_scan_samples, 2)
            else:
                # Odd Line: Separator (2300) -> Porch -> U Scan
                self.add_tone(common.ODD_SEPARATOR_FREQ, self.config.separator_samples)
                self.add_tone(common.PORCH_SEPARATOR_FREQ, self.config.porch_samples)
                self.add_scan_line_tone(u[line], self.config.uv_scan_samples, 1)

        # Concatenate and save
        full_signal = np.concatenate(self.audio_buffer)
        
        # Normalize to 16-bit PCM range
        # full_signal is -1.0 to 1.0
        audio_data = (full_signal * 32767).astype(np.int16)
        
        wavfile.write(file_object, self.config.sample_rate, audio_data)

    def encode(self, output_path):
        with open(output_path, 'wb') as f:
            self.write_wav(f)
        print(f"SSTV Audio saved to {output_path}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Python Robot36 SSTV Encoder")
    parser.add_argument("input_image", help="Path to input image")
    parser.add_argument("output_wav", help="Path to output WAV file")
    parser.add_argument("--width", type=int, default=320, help="Output width (default: 320)")
    parser.add_argument("--height", type=int, default=240, help="Output height (default: 240)")
    
    args = parser.parse_args()
    
    encoder = SSTVEncoder(args.input_image, args.width, args.height)
    encoder.encode(args.output_wav)


In [ ]:
%%writefile decoder.py

import numpy as np
import scipy.signal as signal
from scipy.io import wavfile
from PIL import Image
import common
import sys
import argparse
import subprocess
import os
import tempfile

class SSTVDecoder:
    def __init__(self, audio_source, width=320, height=240, force_device=None, verbose=True, sample_rate_skew=0.0):
        self.verbose = verbose
        self.audio_source = audio_source
        self._load_audio(audio_source)
        
        # Apply Clock Skew (Simulate wow/flutter/playback speed error)
        if sample_rate_skew != 0.0:
            if self.verbose: print(f"Applying Clock Skew: {sample_rate_skew*100:.4f}%")
            self.sample_rate = int(self.sample_rate * (1.0 + sample_rate_skew))
             
        self.freqs = None
        self.config = common.get_mode_config(width, height, self.sample_rate)
        
        # Device Selection for Acceleration
        self.device = self._select_device(force_device)

    def _select_device(self, force=None):
        if force == 'cpu':
            if self.verbose: print("Using Device: CPU (Forced)")
            return None # None triggers CPU fallback block
            
        try:
            import torch
            if force == 'cuda' and torch.cuda.is_available():
                 return torch.device("cuda")
            if force == 'mps' and torch.backends.mps.is_available():
                 return torch.device("mps")
                 
            if torch.cuda.is_available():
                if self.verbose: print("Using Device: CUDA (NVIDIA)")
                return torch.device("cuda")
            elif torch.backends.mps.is_available():
                if self.verbose: print("Using Device: MPS (Apple Silicon)")
                return torch.device("mps")
            else:
                if self.verbose: print("Using Device: CPU (PyTorch)")
                return torch.device("cpu")
        except ImportError:
            if self.verbose: print("Using Device: CPU (NumPy/SciPy) - PyTorch not found")
            return None


    def _load_audio(self, source):
        # Try to read as standard WAV first
        try:
            # wavfile.read accepts file path or file-like object
            self.sample_rate, self.audio_data = wavfile.read(source)
        except ValueError:
            # If source is a string (path), we can try to convert
            if isinstance(source, str):
                if self.verbose: print(f"File {source} is not a standard WAV. Trying to convert via ffmpeg...")
                self._convert_and_load(source)
            else:
                # If it's a file-like object and failed wav read, we can't easily ffmpeg it without writing to disk
                # For now assume file-like objects must be valid WAVs if passed directly
                raise ValueError("Provided audio source is not a valid WAV file and cannot be auto-converted.")
        
        if self.audio_data.ndim > 1:
            self.audio_data = self.audio_data[:, 0] # Take first channel if stereo
        
        # Normalize
        if self.audio_data.dtype == np.int16:
            self.audio_data = self.audio_data / 32768.0
        elif self.audio_data.dtype == np.int32:
             self.audio_data = self.audio_data / 2147483648.0
        elif self.audio_data.dtype == np.uint8:
             self.audio_data = (self.audio_data - 128) / 128.0

    def _convert_and_load(self, path):
        # Use ffmpeg to convert to temporary wav file
        # wavfile.read requires a file on disk (or strictly formatted bytes), 
        # easiest is to dump to temp file
        
        with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tf:
            temp_name = tf.name
            
        try:
            # Convert to 48kHz mono 16-bit PCM wav
            cmd = [
                'ffmpeg', '-y', '-i', path, 
                '-ar', '48000', '-ac', '1', 
                '-f', 'wav', temp_name
            ]
            subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            self.sample_rate, self.audio_data = wavfile.read(temp_name)
        except subprocess.CalledProcessError:
            print("Error: ffmpeg conversion failed or ffmpeg not installed.")
            sys.exit(1)
        finally:
            if os.path.exists(temp_name):
                os.remove(temp_name)
        
    def _hilbert_torch(self, x):
        import torch
        # Hilbert transform via FFT
        # MPS often has issues with arbitrary sizes in older PyTorch versions.
        # Padding to next power of 2 helps stability and performance.
        n_orig = x.shape[-1]
        n_pad = 2**(n_orig - 1).bit_length()
        
        # 1. FFT (with padding)
        # clone() is needed to ensure memory is contiguous which MPS likes
        Xf = torch.fft.fft(x, n=n_pad)
        
        # 2. Create heavy-side function in freq domain
        h = torch.zeros(n_pad, device=x.device)
        if n_pad % 2 == 0:
            h[0] = h[n_pad // 2] = 1
            h[1:n_pad // 2] = 2
        else:
            h[0] = 1
            h[1:(n_pad + 1) // 2] = 2
            
        # 3. Multiply and IFFT
        # We need to truncate back to original length
        return torch.fft.ifft(Xf * h)[..., :n_orig]

    def demodulate(self):
        if self.verbose: print("Demodulating audio...")
        
        if self.device is not None:
             # GPU / PyTorch Acceleration
             import torch
             try:
                 if self.verbose: print(f"Accelerating demodulation on {self.device}...")
                 # Move data to GPU
                 # Ensure float32 for GPU efficiency (audio usually normalized -1..1)
                 audio_tensor = torch.from_numpy(self.audio_data.astype(np.float32)).to(self.device)
                 
                 # 1. Hilbert
                 # PyTorch < 2.0 MPS backend crashes on FFT. 2.0+ supports it.
                 use_cpu_fft = False
                 if self.device.type == 'mps':
                     import re
                     # Simple version parsing
                     v_str = torch.__version__.split('+')[0]
                     major = int(v_str.split('.')[0])
                     if major < 2:
                         use_cpu_fft = True
                 
                 if use_cpu_fft:
                     # Move to CPU for FFT
                     analytic_signal = self._hilbert_torch(audio_tensor.cpu()).to(self.device)
                 else:
                     # CUDA or Newer MPS handles FFT fine
                     analytic_signal = self._hilbert_torch(audio_tensor)
                 
                 # 2. Phase
                 instantaneous_phase = torch.angle(analytic_signal)
                 
                 # 3. Frequency = diff(unwrapped_phase)
                 # Since 'torch.unwrap' is not available in older versions, we calculate distinct phase diffs directly.
                 # diff(unwrap(phi)) is equivalent to (diff(phi) + pi) % (2*pi) - pi
                 
                 diff_phase = torch.diff(instantaneous_phase)
                 # Wrap differences to range [-pi, pi]
                 diff_phase = (diff_phase + np.pi) % (2 * np.pi) - np.pi
                 
                 freqs_gpu = (diff_phase / (2.0 * np.pi) * self.sample_rate)
                 
                 # Move back to CPU for filtering
                 # Concatenate 0 pad like original logic
                 freqs_np = freqs_gpu.cpu().numpy()
                 self.freqs = np.concatenate(([0], freqs_np))
                 
             except Exception as e:
                 print(f"GPU Acceleration failed: {e}. Falling back to CPU.")
                 self.device = None # Fallback for this run
                 
        if self.device is None:
            # CPU Fallback (Original SciPy Logic)
            analytic_signal = signal.hilbert(self.audio_data)
            instantaneous_phase = np.unwrap(np.angle(analytic_signal))
            # f = (1/2pi) * d(phi)/dt
            self.freqs = (np.diff(instantaneous_phase) / (2.0*np.pi) * self.sample_rate)
            # Pad one sample to match length
            self.freqs = np.concatenate(([0], self.freqs))
        
        # Low pass filter using SciPy (CPU)
        # Maintaining CPU filtering ensures exact signal characteristics and stability
        sos = signal.butter(4, 500, 'low', fs=self.sample_rate, output='sos')
        self.freqs = signal.sosfiltfilt(sos, self.freqs)

    def find_sync_pulses(self):
        print("Searching for sync pulses...")
        # Robot36 sync is 1200Hz for 9ms
        # Thresholds
        freq_min = 1100
        freq_max = 1300
        min_duration_samples = common.ms_to_samples(common.SYNC_DURATION_MS * 0.8, self.sample_rate)
        
        is_sync = (self.freqs > freq_min) & (self.freqs < freq_max)
        
        # Find continuous regions
        # diff of boolean gives edges
        edges = np.diff(is_sync.astype(int))
        starts = np.where(edges == 1)[0]
        ends = np.where(edges == -1)[0]
        
        # Handle edge cases
        if len(starts) == 0 or len(ends) == 0:
            return []
            
        if ends[0] < starts[0]:
            ends = ends[1:]
        
        if len(starts) > len(ends):
            starts = starts[:len(ends)]
            
        pulses = []
        for s, e in zip(starts, ends):
            duration = e - s
            if duration >= min_duration_samples:
                # Store the END of the sync pulse (start of the line)
                pulses.append(e)
                
        return pulses

    def decode(self, output_path, shift=0):
        self.demodulate()
        # Flywheel-Based Decoding (Robust to Missed Syncs)
        # 1. Find the first valid sync pulse to bootstrap
        bootstrap_syncs = self.find_sync_pulses()
        if not bootstrap_syncs:
            print("No sync pulses found. Cannot decode.")
            return

        # Calculate samples per line based on total duration of all components
        samples_per_line = (self.config.sync_samples + self.config.sync_porch_samples +
                            self.config.y_scan_samples + self.config.separator_samples +
                            self.config.porch_samples + self.config.uv_scan_samples)

        # Smart Bootstrap: Find the first sync that is part of a periodic chain
        # to avoid locking onto the VIS header.
        start_sync_index = 0
        tolerance = int(samples_per_line * 0.05)
        
        found_chain = False
        for i, candidate in enumerate(bootstrap_syncs):
            # Check for a sync pulse at candidate + samples_per_line
            target = candidate + samples_per_line
            
            # Simple check: is there any sync pulse close to target?
            # Optimization: could use binary search or efficiently scan, but list is small.
            has_next = False
            for s in bootstrap_syncs[i+1:]:
                if abs(s - target) < tolerance:
                    has_next = True
                    break
                if s > target + tolerance:
                    break
            
            if has_next:
                start_sync_index = i
                found_chain = True
                print(f"Smart Bootstrap: Locked onto sync chain at index {i} (Sample {candidate})")
                break
        
        if not found_chain:
            print("Smart Bootstrap: No chain found, using first sync.")

        current_sample_pos = bootstrap_syncs[start_sync_index]
        
        # Format shift output for logging
        shift_str = str(shift)
        if isinstance(shift, (list, np.ndarray)):
            if len(shift) > 5:
                min_s = np.min(shift)
                max_s = np.max(shift)
                mean_s = np.mean(shift)
                shift_str = f"[Random Array len={len(shift)}, Min={min_s}, Max={max_s}, Mean={mean_s:.1f}]"
        
        print(f"Starting decode at sample {current_sample_pos} (Shift: {shift_str})")
        
        # Buffers
        y_image = np.zeros((self.config.height, self.config.width))
        u_image = np.full((self.config.height, self.config.width), 128.0)
        v_image = np.full((self.config.height, self.config.width), 128.0)
        
        # Offsets
        porch_samples = self.config.sync_porch_samples
        y_start = porch_samples
        y_end = y_start + self.config.y_scan_samples
        
        sep_start = y_end
        sep_samples = self.config.separator_samples
        
        porch2_start = sep_start + sep_samples
        porch2_samples = self.config.porch_samples
        
        uv_start = porch2_start + porch2_samples
        uv_end = uv_start + self.config.uv_scan_samples
        
        # State
        last_u = np.full(self.config.width, 128.0)
        last_v = np.full(self.config.width, 128.0)
        
        # Search window for re-sync (e.g. +/- 5% of line width)
        search_window = int(samples_per_line * 0.05)
        
        # Thresholds for re-sync search
        freq_min = 1100
        freq_max = 1300
        min_sync_len = common.ms_to_samples(common.SYNC_DURATION_MS * 0.5, self.sample_rate)

        # Color Phase State
        # Robot36 starts with Line 0 (Even)
        # But if we missed lines at start, we might be on Odd.
        # We start in "Unknown" state? Or just weak lock.
        current_phase_is_odd = False 
        # Start with high mismatch counter to force immediate phase check/flip if signal disagrees
        phase_mismatch_counter = 3 
        
        # Tune Phase Correction
        # We only flip phase if we see strong evidence CONTRARY to current state
        # to avoid noise flipping it.
        
        for line_idx in range(self.config.height):
            # 1. Try to re-sync (Find "Sync End" near current_sample_pos)
            # ... (Sync logic remains same) ...
            
            start_search = max(0, current_sample_pos - search_window)
            end_search = min(len(self.freqs), current_sample_pos + search_window)
            
            best_sync_end = -1
            
            if end_search > start_search:
                local_freqs = self.freqs[start_search:end_search]
                is_sync = (local_freqs > freq_min) & (local_freqs < freq_max)
                
                edges = np.diff(is_sync.astype(int))
                falling_edges = np.where(edges == -1)[0]
                
                if len(falling_edges) > 0:
                     candidates_abs = falling_edges + start_search
                     dists = np.abs(candidates_abs - current_sample_pos)
                     best_idx = np.argmin(dists)
                     
                     edge_pos = candidates_abs[best_idx]
                     check_start =  max(0, edge_pos - min_sync_len)
                     if np.mean(is_sync[(check_start-start_search):(edge_pos-start_search)]) > 0.8:
                         best_sync_end = edge_pos

            if best_sync_end != -1:
                current_sample_pos = best_sync_end

            # 2. Decode the line at current_sample_pos
            # Apply shift for visual alignment (does not affect PLL/Flywheel timing)
            # Support per-line shift (wobble)
            line_shift = shift
            if isinstance(shift, (list, np.ndarray)):
                if line_idx < len(shift):
                    line_shift = shift[line_idx]
                else:
                    line_shift = shift[-1]
            
            effective_start = current_sample_pos + int(line_shift)
            
            # Use effective_start as base for all extraction
            sync_end = effective_start
            
            # Check limits
            if sync_end + uv_end >= len(self.freqs):
                break

            # Y Channel
            y_freqs = self.freqs[sync_end + y_start : sync_end + y_end]
            if len(y_freqs) > 0:
                y_pixels_freq = signal.resample(y_freqs, self.config.width)
                y_vals = (y_pixels_freq - common.BLACK_FREQ) * 255.0 / (common.WHITE_FREQ - common.BLACK_FREQ)
                y_image[line_idx] = y_vals
            
            # Color Mode Detection & Phase Locking
            # Expected frequencies: Even=1500Hz, Odd=2300Hz
            # Separator is located after Y scan
            
            margin = int(sep_samples * 0.25)
            curr_sep_start = int(sync_end + sep_start + margin)
            curr_sep_end = int(sync_end + sep_start + sep_samples - margin)
            
            # Measure actual separator frequency
            measured_freq = 0
            if curr_sep_end < len(self.freqs):
                sep_freqs = self.freqs[curr_sep_start : curr_sep_end]
                if len(sep_freqs) > 0:
                    measured_freq = np.mean(sep_freqs)
            
            # Phase Correction Logic (Hysteresis)
            # Only correct if we are VERY confident.
            # Even (1500) <-> Odd (2300). Midpoint ~1900.
            # Strong Even: < 1700. Strong Odd: > 2100.
            
            strong_odd = measured_freq > 2100
            strong_even = measured_freq > 0 and measured_freq < 1700
            
            mismatch_detected = False
            
            if current_phase_is_odd:
                if strong_even:
                    mismatch_detected = True
            else:
                if strong_odd:
                    mismatch_detected = True
            
            if mismatch_detected:
                phase_mismatch_counter += 1
            else:
                # Decay the counter if signal agrees or is ambiguous (noise)
                # We decay slowly aka we require a few good lines to trust stability?
                # Or fast decay? Fast decay is better to recover from false positives.
                phase_mismatch_counter = max(0, phase_mismatch_counter - 1)
                
            if phase_mismatch_counter >= 3:
                # 3 lines of strong contrary evidence -> Flip Phase
                current_phase_is_odd = not current_phase_is_odd
                phase_mismatch_counter = 0
            
            # UV Channel
            uv_freqs = self.freqs[sync_end + uv_start : sync_end + uv_end]
            if len(uv_freqs) > 0:
                uv_pixels_freq = signal.resample(uv_freqs, self.config.width)
                uv_vals = (uv_pixels_freq - common.BLACK_FREQ) * 255.0 / (common.WHITE_FREQ - common.BLACK_FREQ)
                
                if current_phase_is_odd:
                     # Odd Line -> U Scan
                     last_u = uv_vals
                     # V uses previous
                else:
                     # Even Line -> V Scan
                     last_v = uv_vals
                     # U uses previous
            
            u_image[line_idx] = last_u
            v_image[line_idx] = last_v
            
            # 3. Advance Clock & Phase
            current_sample_pos += samples_per_line
            current_phase_is_odd = not current_phase_is_odd
        
        # Convert to RGB
        r, g, b = common.yuv_to_rgb(y_image, u_image, v_image)
        
        rgb_img = np.stack((r, g, b), axis=-1).astype(np.uint8)
        img = Image.fromarray(rgb_img)
        img.save(output_path)
        print(f"Decoded image saved to {output_path}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Python Robot36 SSTV Decoder")
    parser.add_argument("input_file", help="Path to input audio file (WAV, MP3, etc.)")
    parser.add_argument("output_image", help="Path to output image")
    parser.add_argument("--width", type=int, default=320, help="Input width (default: 320)")
    parser.add_argument("--height", type=int, default=240, help="Input height (default: 240)")
    parser.add_argument("--shift", type=int, default=0, help="Phase shift in samples (default: 0)")
    
    args = parser.parse_args()
    
    decoder = SSTVDecoder(args.input_file, args.width, args.height)
    decoder.decode(args.output_image, shift=args.shift)


In [ ]:
%%writefile sstv_filter.py

import os
import sys
import argparse
import io
import subprocess
import numpy as np
from PIL import Image

# Add current directory to path to import encoder/decoder
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

from encoder import SSTVEncoder
from decoder import SSTVDecoder
import common

def get_resolution_from_preset(preset):
    presets = {
        'default': (320, 240),
        'ntsc': (720, 480),
        'pal': (720, 576),
        'robot36': (320, 240),
        'martin1': (320, 256),
        'scott1': (320, 256),
        '144p': (256, 144),
        '240p': (426, 240),
        '360p': (640, 360),
        '480p': (854, 480),
        '720p': (1280, 720),
        '1080p': (1920, 1080),
    }
    return presets.get(preset.lower(), (320, 240))



def apply_compression(wav_bytes, bitrate):
    """
    Compresses WAV bytes (PCM) to MP3/Opus using ffmpeg, then decompresses back to WAV bytes.
    This simulates the transmission loss.
    """
    try:
        # Determine codec and container based on bitrate
        # MP3 standard floor is often 8kbps or 32kbps depending on sample rate.
        # Opus can go very low (1k).
        
        # Extract numerical value from bitrate string
        bk = int(''.join(filter(str.isdigit, bitrate)))
        
        codec = 'libmp3lame'
        fmt = 'mp3'
        extra_args = []
        
        if bk < 4:
            # Use Speex for extremely low bitrates (< 6k)
            # Opus hits a floor ~6k. Speex can go down to ~2k (Narrowband).
            codec = 'libspeex'
            fmt = 'ogg'
            # Speex NB requires 8k sample rate
            # -vad 0 disables VAD to ensure constant transmission if possible (though speex is inherently VBR-ish)
            extra_args = ['-ar', '8000']
        elif bk < 8:
            # MP3 typically doesn't support < 8k. Use Opus.
            codec = 'libopus'
            fmt = 'opus'
            # Low bitrate Opus usually wants 48k (internal) but acceptable.
            # Add voip application tune for extreme low bitrates
            extra_args = ['-application', 'voip', '-ar', '8000']
        elif bk < 32:
            # MP3 at <32k often requires lower sample rate (16k, 22.05k, 24k)
            # 48k input might cause encoder failure.
            # Let's force resampling to 16k for these intermediate bitrates
            extra_args = ['-ar', '16000']
            
        cmd_enc = [
            'ffmpeg', '-y', '-f', 'wav', '-i', 'pipe:0',
            '-c:a', codec, '-b:a', bitrate
        ] + extra_args + ['-f', fmt, 'pipe:1']
        
        # 1. Encode
        process_enc = subprocess.Popen(
            cmd_enc, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
        compressed_data, enc_err = process_enc.communicate(input=wav_bytes)
        
        if process_enc.returncode != 0:
            print(f"Warning: Compression encoding failed. Using original audio.")
            print(f"FFmpeg Error: {enc_err.decode('utf-8', errors='ignore')}")
            return wav_bytes

        # 2. Decode back to WAV (PCM)
        # Note: ffmpeg auto-detects codec from content usually, but pipe might need hint if no header?
        # MP3/Opus in pipe usually has headers.
        cmd_dec = [
            'ffmpeg', '-y', '-i', 'pipe:0',
            '-f', 'wav', 'pipe:1'
        ]
        
        process_dec = subprocess.Popen(
            cmd_dec, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
        wav_pcm, dec_err = process_dec.communicate(input=compressed_data)
        
        if process_dec.returncode != 0:
            print(f"Warning: Compression decoding failed. Using original audio.")
            print(f"FFmpeg Decode Error: {dec_err.decode('utf-8', errors='ignore')}")
            return wav_bytes
            
        return wav_pcm

    except Exception as e:
        print(f"Error during compression: {e}")
        return wav_bytes

def main():
    parser = argparse.ArgumentParser(description="SSTV Image Filter Tool")
    parser.add_argument("input_image", help="Input image path")
    parser.add_argument("output_image", help="Output image path")
    
    # Resolution arguments
    group = parser.add_mutually_exclusive_group()
    group.add_argument("--preset", help="Resolution preset (default, ntsc, pal, 144p, etc.)", default="default")
    group.add_argument("--res", help="Custom resolution WxH (e.g. 640x480)")
    
    # Compression
    parser.add_argument("-k", "--bitrate", help="Audio bitrate (e.g. 320k, 64k, 1k)", default=None)
    
    # Save intermediate audio
    parser.add_argument("--save-audio", help="Path to save the generated audio WAV file", default=None)

    args = parser.parse_args()
    
    # Determine resolution
    width, height = 320, 240
    if args.res:
        try:
            w, h = map(int, args.res.split('x'))
            width, height = w, h
        except:
            print("Invalid resolution format. Using default.")
    else:
        width, height = get_resolution_from_preset(args.preset)
        
    print(f"Processing image: {args.input_image}")
    print(f"Target Resolution: {width}x{height}")
    
    # 1. Encode
    print("Encoding to SSTV...")
    encoder = SSTVEncoder(args.input_image, width, height)
    
    # In-memory WAV buffer
    wav_buffer = io.BytesIO()
    encoder.write_wav(wav_buffer)
    wav_bytes = wav_buffer.getvalue()
    
    # 2. Compress (Optional)
    if args.bitrate:
        print(f"Applying compression ({args.bitrate})...")
        wav_bytes = apply_compression(wav_bytes, args.bitrate)
        
    # Save audio if requested
    if args.save_audio:
        with open(args.save_audio, 'wb') as f:
            f.write(wav_bytes)
        print(f"Audio saved to {args.save_audio}")
        
    # 3. Decode
    print("Decoding SSTV...")
    # Wrap bytes in BytesIO for wavfile.read
    decode_buffer = io.BytesIO(wav_bytes)
    
    decoder = SSTVDecoder(decode_buffer, width, height)
    decoder.decode(args.output_image)
    
    print(f"Finished. Saved to {args.output_image}")

if __name__ == "__main__":
    main()


In [ ]:
%%writefile video_sstv.py

import cv2
import argparse
import sys
import os
import multiprocessing
import tempfile
import subprocess
import shutil
import io
import time
from concurrent.futures import ProcessPoolExecutor
import numpy as np
from PIL import Image

# Import local modules
sys.path.append(os.path.dirname(os.path.abspath(__file__)))
from encoder import SSTVEncoder
from decoder import SSTVDecoder
from sstv_filter import get_resolution_from_preset, apply_compression
import common

def process_frame_task(args):
    """
    Worker function to process a single frame.
    args: (frame_bgr, frame_idx, config_dict)
    Returns: (frame_idx, processed_frame_bgr)
    """
    frame_bgr, frame_idx, config = args
    
    try:
        # 1. Resize to Target SSTV Resolution
        target_w, target_h = config['target_size']
        original_h, original_w = frame_bgr.shape[:2]
        
        # Convert BGR to RGB (PIL)
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(frame_rgb)
        
        # Resize to SSTV res
        # Save to temp file for Encoder (Encoder takes path) -> Optimization: Encoder should take PIL Image?
        # Looking at encoder.py, it takes image_path. 
        # We should modify encoder to accept image object or save temp.
        # For now, save temp to be safe standard.
        
        # Unique temp file for this worker (Only for intermediate WAV/PNG if needed)
        pid = multiprocessing.current_process().pid
        # temp_in = os.path.join(config['temp_dir'], f"frame_{frame_idx}_{pid}_in.png") # Removed: In-memory
        temp_out_img = os.path.join(config['temp_dir'], f"frame_{frame_idx}_{pid}_out.png")
        
        # pil_img.save(temp_in) # Removed
        
        # 2. Encode
        # Encoder accepts PIL Image directly now!
        encoder = SSTVEncoder(pil_img, target_w, target_h, sample_rate=48000)
        
        # Write WAV to memory buffer to avoid disk I/O if possible? 
        # encoder.write_wav takes file_object.
        wav_buffer = io.BytesIO()
        encoder.write_wav(wav_buffer)
        wav_bytes = wav_buffer.getvalue()
        
        # 3. Compress (Simulate Channel)
        if config['bitrate']:
            wav_bytes = apply_compression(wav_bytes, config['bitrate'])
            
        # 4. Decode
        # Decoder takes file-like object
        # Suppress WavFileWarning from scipy
        import warnings
        from scipy.io import wavfile
        warnings.filterwarnings("ignore", category=wavfile.WavFileWarning)
        
        # Calculate Random parameters for this frame
        # Seed = Base + FrameIdx
        rng = np.random.RandomState(config['base_seed'] + frame_idx)
        
        # 1. Random Shift (Global Phase Jitter)
        # Add to static shift
        base_shift = config['shift']
        if config['rand_shift_range'] > 0:
            jitter = rng.randint(-config['rand_shift_range'], config['rand_shift_range'] + 1)
            base_shift += jitter
            
        # 1.5 Random Wobble (Line Tearing)
        if config['rand_wobble_max'] > 0:
            # Generate array of random shifts per line
            wobble = rng.randint(-config['rand_wobble_max'], config['rand_wobble_max'] + 1, size=target_h)
            frame_shift = wobble + base_shift
        else:
            frame_shift = base_shift
            
        # 2. Random Skew (Clock Drift)
        # sample_rate_skew: e.g. 0.0005
        frame_skew = 0.0
        if config['rand_skew_max'] > 0:
            frame_skew = rng.uniform(-config['rand_skew_max'], config['rand_skew_max'])
        
        decode_buffer = io.BytesIO(wav_bytes)
        # Pass skew to decoder
        decoder = SSTVDecoder(
            decode_buffer, target_w, target_h, 
            force_device=config['device'], 
            verbose=False,
            sample_rate_skew=frame_skew
        )
        
        # Decoder.decode saves to file. 
        # decoder.decode calls demodulate
        decoder.decode(temp_out_img, shift=frame_shift)
        
        # 5. Read back and Resize to Output (Original or Target?)
        # Usually video filters output same resolution as input unless specified.
        # But feasibility report said: "Resize(1080p) -> Output".
        # So we resize back to ORIGINAL video resolution.
        
        if os.path.exists(temp_out_img):
            processed_pil = Image.open(temp_out_img).convert('RGB')
            # Resize back to original video size
            processed_pil = processed_pil.resize((original_w, original_h), Image.Resampling.NEAREST) # Nearest for glitch look? Or Lanczos?
            # Let's use Bilinear or Lanczos for better quality upscaling.
            processed_pil = processed_pil.resize((original_w, original_h), Image.Resampling.LANCZOS)
            
            processed_rgb = np.array(processed_pil)
            processed_bgr = cv2.cvtColor(processed_rgb, cv2.COLOR_RGB2BGR)
        else:
            # Fallback
            processed_bgr = frame_bgr

        # Cleanup
        try:
            # temp_in is removed/in-memory now
            if os.path.exists(temp_out_img): os.remove(temp_out_img)
        except:
            pass
            
        return (frame_idx, processed_bgr)
        
    except Exception as e:
        print(f"Error processing frame {frame_idx}: {e}")
        return (frame_idx, frame_bgr) # Return original on failure

def main():
    parser = argparse.ArgumentParser(description="Convert Video to SSTV Style")
    parser.add_argument("input_video", help="Path to input video file")
    parser.add_argument("output_video", nargs='?', help="Path to output video file (optional if --no-video is used)")
    
    # SSTV Config
    parser.add_argument("--preset", default="default", help="SSTV Mode Preset (default, ntsc, 1080p etc)")
    parser.add_argument("--width", type=int, help="Override SSTV processing width")
    parser.add_argument("--height", type=int, help="Override SSTV processing height")
    parser.add_argument("--shift", type=int, default=0, help="Phase shift (horizontal alignment)")
    parser.add_argument("-k", "--bitrate", help="Audio bitrate simulation (e.g. 32k, 8k, 4k)")
    
    # Video Config
    parser.add_argument("--fps", type=float, help="Output FPS (default: same as input)")
    parser.add_argument("--no-audio", action="store_true", help="Do not copy audio from input")
    parser.add_argument("--no-video", action="store_true", help="Do not generate output video file (process frames only)")
    parser.add_argument("--workers", type=int, default=multiprocessing.cpu_count(), help="Number of parallel workers")
    parser.add_argument("--device", choices=['cpu', 'cuda', 'mps'], default=None, help="Force acceleration device")
    
    parser.add_argument('--frames-dir', type=str, help="Directory to save processed frames. Defaults to './<video_name>_frames'.")
    parser.add_argument('--clean-frames', action='store_true', help="Delete frames directory after processing.")
    parser.add_argument('--start', type=int, default=0, help="Start frame index.")
    parser.add_argument('--end', type=int, default=None, help="End frame index.")
    parser.add_argument('--index-mode', type=str, choices=['input', 'output'], default='input', help="Whether start/end indices refer to 'input' (original) or 'output' (target) video frames.")
    
    parser.add_argument('--random-shift-range', type=int, default=0, help="Randomize horizontal shift per frame by +/- N pixels (Global Jitter).")
    parser.add_argument('--random-wobble-max', type=int, default=0, help="Randomize horizontal shift per scanline by +/- N pixels (Line Tearing).")
    parser.add_argument('--random-skew', type=float, default=0.0, help="Randomize sample rate skew per frame by +/- F (e.g. 0.0005) (Clock Drift).")
    parser.add_argument('--seed', type=int, default=None, help="Random seed for reproducible effects.")
    
    args = parser.parse_args()

    if not args.output_video and not args.no_video:
        parser.error("the following arguments are required: output_video (unless --no-video is specified)")
    
    print(f"--- Video SSTV Processing ---")
    print(f"Input: {args.input_video}")
    # Resolution parsed later
    print(f"Bitrate: {args.bitrate if args.bitrate else 'Lossless/PCM'}")
    print(f"Workers: {args.workers}")
    print(f"Device: {args.device if args.device else 'Auto'}")
    
    # 1. Open Video
    cap = cv2.VideoCapture(args.input_video)
    if not cap.isOpened():
        print("Error: Could not open video.")
        sys.exit(1)
        
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width_video = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height_video = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    target_fps = args.fps if args.fps else original_fps
    
    # Smart Resolution Logic
    # 0 -> Keep Original
    # -1 (or negative) -> Scale proportionally
    # Both <= 0 -> Keep Original
    
    base_w, base_h = get_resolution_from_preset(args.preset)
    
    # Start with base or overrides
    # If args.width is None (not provided), use base. If provided (0, -1, 100), use arg.
    req_w = args.width if args.width is not None else (base_w if args.preset != 'default' else 0) 
    req_h = args.height if args.height is not None else (base_h if args.preset != 'default' else 0)
    
    # If preset is default and no args, args.width is None, base is 320.
    # Current logic: args.preset default is "default".
    # We want: 
    # If user didn't specify preset OR width/height -> Default 320x240 (Robot36ish)
    # If user specified preset -> use preset res
    # If user specified width/height -> override
    
    # If overrides exist, they take precedence.
    raw_w = args.width if args.width is not None else None
    raw_h = args.height if args.height is not None else None
    
    if raw_w is None and raw_h is None:
        # Pure preset mode
        target_w, target_h = base_w, base_h
    else:
        # Custom / Override mode
        w_val = raw_w if raw_w is not None else 0
        h_val = raw_h if raw_h is not None else 0
        
        ar_video = width_video / height_video
        
        # 1. Resolve '0' to Original Dimension
        # 2. Mark '<0' as specific flag for calculation
        
        final_w = w_val
        if w_val == 0:
            final_w = width_video
        
        final_h = h_val
        if h_val == 0:
            final_h = height_video
            
        # 3. Handle Auto-Scale Flags (<0)
        # Note: final_w/h are now either >0 or <0. They cannot be 0.
        
        if final_w < 0 and final_h < 0:
             # Both -1 -> Keep Original
             target_w, target_h = width_video, height_video
        elif final_w < 0:
             # Width is -1, Height is concrete (>0)
             # Scale width to match height using aspect ratio
             # w = h * ar
             target_h = final_h
             target_w = int(final_h * ar_video)
        elif final_h < 0:
             # Height is -1, Width is concrete (>0)
             # Scale height to match width
             # h = w / ar
             target_w = final_w
             target_h = int(final_w / ar_video)
        else:
             # Both concrete
             target_w, target_h = final_w, final_h

    # Ensure even dimensions (video codecs prefer them)
    if target_w % 2 != 0: target_w += 1
    if target_h % 2 != 0: target_h += 1

    print(f"Video Info: {width_video}x{height_video} @ {original_fps}fps ({total_frames} frames)")
    print(f"Target Resolution: {target_w}x{target_h}")
    
    # Create Frames Directory
    if args.frames_dir:
        temp_dir = args.frames_dir
    else:
        # Default: current_dir / <video_name>_frames
        base_name = os.path.splitext(os.path.basename(args.input_video))[0]
        temp_dir = os.path.join(os.getcwd(), f"{base_name}_frames")
    
    os.makedirs(temp_dir, exist_ok=True)
    print(f"Frames directory: {temp_dir}")
    
    # 2. Extract Audio (if needed)
    audio_path = None
    # Frame Range Logic
    fps_ratio = original_fps / target_fps
    
    start_frame_input = args.start
    end_frame_input = args.end if args.end is not None else total_frames
    
    if args.index_mode == 'output':
        # Convert output index to input index
        start_frame_input = int(start_frame_input * fps_ratio)
        end_frame_input = int(end_frame_input * fps_ratio)
        
    start_frame_input = max(0, start_frame_input)
    end_frame_input = min(int(total_frames), end_frame_input)
    
    print(f"Processing range (Input Indices): {start_frame_input} to {end_frame_input}")

    # 2. Extract Audio (Segment)
    audio_path = os.path.join(temp_dir, "extracted_audio.wav")
    
    start_time_sec = start_frame_input / original_fps
    duration_sec = (end_frame_input - start_frame_input) / original_fps
    
    if not args.no_audio:
        print(f"Extracting audio segment: Start={start_time_sec:.2f}s, Dur={duration_sec:.2f}s")
        try:
            # -ss before -i is fast seek (keyframe), but less accurate?
            # input is video.
            # -ss after -i is frame accurate decoding.
            # We want logic: input -> extract -ss ... -t ...
            cmd = [
                'ffmpeg', '-y', '-i', args.input_video, 
                '-ss', str(start_time_sec),
                '-t', str(duration_sec),
                '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', audio_path
            ]
            subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except Exception as e:
            print(f"Warning: Audio extraction failed: {e}")
            audio_path = None
    else:
        audio_path = None

    # Seek Video
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_input)
    
    # 3. Process Frames
    # 3. Process Frames
    frames_dir = temp_dir
    # os.makedirs(frames_dir, exist_ok=True) # Already created
    
    # Random Base Seed
    if args.seed is not None:
        base_seed = args.seed
    else:
        base_seed = int(time.time())
    print(f"Random Seed: {base_seed}")

    config = {
        'target_size': (target_w, target_h),
        'temp_dir': temp_dir,
        'bitrate': args.bitrate,
        'shift': args.shift,
        'device': args.device,
        # Random Config
        'rand_shift_range': args.random_shift_range,
        'rand_wobble_max': args.random_wobble_max,
        'rand_skew_max': args.random_skew,
        'base_seed': base_seed
    }
    
    
    read_count = start_frame_input
    process_count = 0 
    
    pool = multiprocessing.Pool(processes=args.workers)
    results = []
    
    # Buffer for async results (index, object)
    pending_results = {} # index -> AsyncResult
    next_write_idx = 0
    
    acc_error = 0.0
    
    try:
        while True:
            if read_count >= end_frame_input:
                break
                
            ret, frame = cap.read()
            if not ret:
                break
            
            # Decimation Logic
            process_this = False
            if args.fps:
                acc_error += 1
                if acc_error >= frame_interval:
                    process_this = True
                    acc_error -= frame_interval
            else:
                process_this = True
                
            if process_this:
                # Dispatch
                idx = process_count
                
                # Apply processing asynchronously
                res = pool.apply_async(process_frame_task, args=((frame, idx, config),))
                pending_results[idx] = res
                process_count += 1
                
                # Flow control: don't OOM
                # Check for finished tasks dynamically
                while len(pending_results) > args.workers * 2:
                    # Find any ready task
                    finished_indices = []
                    for p_idx, p_res in pending_results.items():
                        if p_res.ready():
                            r_idx, r_frame = p_res.get()
                            
                            # Save Immediate
                            # Use r_idx (which tracks 0, 1, 2...) for filename
                            out_path = os.path.join(frames_dir, f"frame_{r_idx:05d}.png")
                            success = cv2.imwrite(out_path, r_frame)
                            if not success:
                                print(f"Error: Failed to write {out_path}")
                                
                            finished_indices.append(p_idx)
                            print(f"Processed frame {r_idx+1}/...", end='\r')
                    
                    # Remove finished
                    for p_idx in finished_indices:
                        del pending_results[p_idx]
                    
                    # If still too many, sleep brief
                    if len(pending_results) > args.workers * 2:
                        time.sleep(0.01) # Brief yield
                        
            read_count += 1
            
        # Drain rest
        pool.close()
        pool.join()
        
        # Collect remaining (all should be ready now)
        for p_idx, p_res in pending_results.items():
             r_idx, r_frame = p_res.get()
             
             out_path = os.path.join(frames_dir, f"frame_{r_idx:05d}.png")
             success = cv2.imwrite(out_path, r_frame)
             if not success:
                 print(f"Error: Failed to write {out_path}")
             print(f"Processed frame {r_idx+1}/{process_count}", end='\r')
        
        # Clear buffer
        pending_results.clear()
    
    except KeyboardInterrupt:
        print("\nStopping...")
        pool.terminate()
        
    finally:
        cap.release()
        print("\nFrames extraction complete.")
        
    # 4. Assemble Video with FFmpeg
    if not args.no_video and args.output_video:
        print("Assembling video...")
        final_output = args.output_video
        
        # FFmpeg command to assemble frames + audio
        # -framerate must be before -i image
        assemble_cmd = [
            'ffmpeg', '-y', 
            '-framerate', str(target_fps),
            '-start_number', '0',
            '-i', os.path.join(frames_dir, 'frame_%05d.png')
        ]
        
        if audio_path and os.path.exists(audio_path):
            assemble_cmd += ['-i', audio_path, '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-c:a', 'aac', '-shortest', final_output]
        else:
            assemble_cmd += ['-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-an', final_output]
            
        subprocess.run(assemble_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)
        print(f"\nDone! Video saved: {final_output}")
    else:
        print("\nSkipping video assembly (--no-video selected or no output file).")
    
    # Cleanup
    if args.clean_frames:
        print(f"Cleaning up frames directory: {temp_dir}")
        shutil.rmtree(temp_dir, ignore_errors=True)
    else:
        print(f"Frames saved in: {temp_dir}")

if __name__ == "__main__":
    main()


In [ ]:
#@title 3. Setup Input Source
import os
from google.colab import files
from google.colab import drive

source_type = "Mount Google Drive" #@param ["Mount Google Drive", "Upload File"]

if source_type == "Mount Google Drive":
    drive.mount('/content/drive')
    print("Google Drive mounted at /content/drive")
elif source_type == "Upload File":
    uploaded = files.upload()
    if uploaded:
        fname = list(uploaded.keys())[0]
        print(f"File uploaded to: /content/{fname}")
        print("Please copy this path to the 'input_path' field below.")


In [ ]:
#@title 4. Run Process

input_path = "/content/drive/MyDrive/videos/test.mp4" #@param {type:"string"}
output_filename = "/content/drive/MyDrive/videos/output_sstv.mp4" #@param {type:"string"}

#@markdown ### Basic Settings
preset = "ntsc" #@param ["ntsc", "pal", "robot36", "martin1", "scott1", "144p", "240p", "360p", "480p", "720p", "1080p", "Custom"]
custom_width = -1 #@param {type:"integer", help:"Only used if preset is Custom. 0=Original, -1=Scale by Ratio"}
custom_height = -1 #@param {type:"integer", help:"Only used if preset is Custom. 0=Original, -1=Scale by Ratio"}
bitrate = "12k" #@param ["Lossless", "320k", "256k", "192k", "128k", "96k", "64k", "48k", "32k", "24k", "16k", "12k", "8k", "4k", "2k"]
workers = 4 #@param {type:"integer"}
no_video = False #@param {type:"boolean"}

#@markdown ### Random Glitch Effects
random_shift_range = 5 #@param {type:"integer", help:"Horizontal shake amplitude (pixels)"}
random_wobble_max = 3 #@param {type:"integer", help:"Scanline tearing/wobble amplitude (pixels)"}
random_skew = 0.0005 #@param {type:"number", help:"Clock drift/slant factor (e.g. 0.0005)"}
seed = 42 #@param {type:"integer", help:"Random seed for reproducibility. Set -1 for random."}

#@markdown ### Advanced Range (Optional)
use_frame_range = False #@param {type:"boolean"}
start_frame = 0 #@param {type:"integer"}
end_frame = 0 #@param {type:"integer"}

import subprocess

if not os.path.exists(input_path):
    print(f"Error: Input file not found at {input_path}\nPlease check the path or mount Google Drive.")
else:
    print(f"Processing: {input_path}")
    cmd = ["python", "-u", "video_sstv.py", input_path]
    
    if not no_video:
        cmd.append(output_filename)
        
    # Basic flags
    if preset == "Custom":
        cmd.extend(["--width", str(custom_width), "--height", str(custom_height)])
    else:
        cmd.extend(["--preset", preset])
        
    if bitrate != "Lossless":
        cmd.extend(["--bitrate", bitrate])
    cmd.extend(["--workers", str(workers)])
    
    if no_video:
        cmd.append("--no-video")
        
    # Random flags
    if random_shift_range > 0:
        cmd.extend(["--random-shift-range", str(random_shift_range)])
    if random_wobble_max > 0:
        cmd.extend(["--random-wobble-max", str(random_wobble_max)])
    if random_skew > 0:
        cmd.extend(["--random-skew", str(random_skew)])
    if seed != -1:
        cmd.extend(["--seed", str(seed)])
        
    # Range flags
    if use_frame_range:
        cmd.extend(["--start", str(start_frame)])
        if end_frame > 0:
             cmd.extend(["--end", str(end_frame)])
             
    print(f"Executing: {' '.join(cmd)}")
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)
    for line in iter(process.stdout.readline, ''):
        print(line, end='')
    process.wait()
    if process.returncode != 0:
        print(f"Error: Process failed with exit code {process.returncode}")
    else:
        print("Processing Complete.")


In [ ]:
#@title 5. Download Output
from google.colab import files
import os

if os.path.exists(output_filename):
    if "/content/drive" in output_filename:
        print(f"File saved to Google Drive: {output_filename}\n(No download needed)")
    else:
        files.download(output_filename)
else:
    print(f"File {output_filename} not found. Did the process fail?")